In [1]:
import pandas as pd
import io
import re
import csv
import json
import numpy as np
from numpy import savetxt, loadtxt, asarray
from tqdm import tqdm
import math
import ast
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import seaborn as sns
import keras
from keras import backend as K
from progressbar import ProgressBar
import matplotlib.pyplot as plt
from keras import Sequential, Model
from random import randrange
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Input
import time
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.externals import joblib



from plot_figures import save_model

pbar = ProgressBar()
tqdm.pandas()

Using TensorFlow backend.


In [2]:


max_blocks = 37 #(36 + 1 FC layer at the end)
nb_param =7
nb_hw_param = 12

inversed = True
only_inversed = False

if inversed and not only_inversed:
    path_entire_model= "/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/results_best_models/multiply/with_inversed_hw"
elif only_inversed:
    path_entire_model= "/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/results_best_models/multiply/only_inversed_hw"
else : 
    path_entire_model= "/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/results_best_models/multiply/no_inversed_hw"

BATCH_SIZE = 2048
SHUFFLE_BUFFER_SIZE = 100

DATASET_SIZE = 168308
train_size = int(0.90 * DATASET_SIZE) #135 805
test_size = int(0.10 * DATASET_SIZE) # 15 089

nb_training_batches = train_size //BATCH_SIZE +1
nb_test_batches = test_size //BATCH_SIZE +1

In [3]:

def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.asarray(ast.literal_eval(array_string))

# Load Proccessed Neural Networks (without HyperParameters)

In [4]:
path_raw_val = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/good_data/parsed_nondups_val_3.csv'
path_processed_val_nn = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/good_data/val7_from1600.csv'




In [5]:
#%pycache
tin= time.clock()   
val = pd.read_csv(path_processed_val_nn,converters={'NN_dataframe': from_np_array})
tfin= time.clock()
print(f'Total time : {(tfin-tin)/60} min')

Total time : 0.15731295000000003 min


# Complete to zeros (max shape)

In [6]:
def get_shape(arr):
    return arr.shape[0]

def add_zero_blocks(arr):
    zero_blocks = np.zeros((max_blocks-arr.shape[0],arr.shape[1]))
    return np.append(arr, zero_blocks, axis=0)

val["nb_blocks"] = val["NN_dataframe"].apply(lambda x : get_shape(x))
val['NN_dataframe']= val['NN_dataframe'].apply(lambda x : add_zero_blocks(x))




# Get y_train, y_val


In [7]:
# To retrieve y_train / y_test

#train_hw = pd.read_csv(path_raw_train)
val_hw = pd.read_csv(path_raw_val)

#y_train = np.array(train_hw["total_power"].tolist())
#y_train=y_train.reshape(y_train.shape[0],-1)

y_val = np.array(val_hw["total_power"].tolist())
y_val=y_val.reshape(y_val.shape[0],-1)

--------
# HW PARAM ALONE

In [8]:
#X_train_hw_param  = train_hw[['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size']]
#X_train_hw_param['tot_mac'] = X_train_hw_param['mac_num']*X_train_hw_param['mac_array_num']

X_val_hw_param  = val_hw[['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size']]
X_val_hw_param['tot_mac'] = X_val_hw_param['mac_num']*X_val_hw_param['mac_array_num']

#X_train_hw_param.head()

# Shapes of the blocks

In [9]:
#X_train_shape = np.array(train["nb_blocks"])
X_val_shape = np.array(val["nb_blocks"])

#X_val_shape

# Inv

In [10]:
def inv(x):
    return 1/(1+x)


if inversed : 
#nb_hw_param = 6
    nb_hw_param = 12

    X_val_hw_param['1/mac_num'] = X_val_hw_param['mac_num'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/mac_array_num'] = X_val_hw_param['mac_array_num'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/data_bits'] = X_val_hw_param['data_bits'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/sram_size'] = X_val_hw_param['sram_size'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/max_filter_size'] = X_val_hw_param['max_filter_size'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/tot_mac'] = X_val_hw_param['tot_mac'].progress_apply(lambda x : inv(x))



100%|██████████| 24274/24274 [00:00<00:00, 386300.30it/s]


In [11]:
X_val_hw_param.head()

,mac_num,mac_array_num,data_bits,sram_size,max_filter_size,tot_mac,1/mac_num,1/mac_array_num,1/data_bits,1/sram_size,1/max_filter_size,1/tot_mac
0,37.0,2.0,128.0,199936.0,4098.0,74.0,0.026316,0.333333,0.007752,0.000005,0.000244,0.013333
1,91.0,2.0,256.0,119936.0,2048.0,182.0,0.010870,0.333333,0.003891,0.000008,0.000488,0.005464
2,74.0,2.0,256.0,139904.0,1024.0,148.0,0.013333,0.333333,0.003891,0.000007,0.000976,0.006711
3,70.0,2.0,512.0,80000.0,512.0,140.0,0.014085,0.333333,0.001949,0.000012,0.001949,0.007092
4,103.0,2.0,256.0,199936.0,4098.0,206.0,0.009615,0.333333,0.003891,0.000005,0.000244,0.004831


## HW Normalize

In [12]:

scaler = joblib.load('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/params/hw_scaler.pkl') 
X_val_hw_param_norm = scaler.transform(X_val_hw_param )



In [13]:
    hw_array_test = np.array([1.023785822261557565e-01,-4.262310244296395600e-01,2.295995016604397698e-01,-4.441886177106757483e-01,-8.459185226109141587e-01,-2.938802963497538223e-01,-3.464242712811745339e-01,4.572050514380619490e-01,-6.563799113750447001e-01,3.803529224492657179e-01,1.081335456679819673e+00,-2.117873165907936950e-01])
    hw_array_test = hw_array_test.reshape((1, *hw_array_test.shape))
    hw_array_test = tf.convert_to_tensor(hw_array_test, dtype=tf.float32)   

In [14]:
scaler.inverse_transform(hw_array_test)

array([[7.5964783e+01, 2.4868679e+00, 8.3354712e+02, 9.3668180e+04,
        6.4995160e+02, 1.8534184e+02, 1.9902330e-02, 3.0593166e-01,
        1.6979771e-03, 1.2175874e-05, 1.6928774e-03, 8.8932756e-03]],
      dtype=float32)

In [15]:
pd.DataFrame(X_val_hw_param_norm).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.595863,-0.489793,-0.841296,0.754353,2.061669,-0.412974,-0.169928,0.706748,1.441544,-1.095801,-1.155747,0.067051
1,0.371807,-0.489793,-0.647015,-0.147927,0.332994,-0.297455,-0.595003,0.706748,0.103602,-0.409372,-0.778893,-0.427118
2,0.067170,-0.489793,-0.647015,0.077282,-0.530500,-0.333822,-0.527201,0.706748,0.103602,-0.654222,-0.026105,-0.348810
3,-0.004509,-0.489793,-0.258452,-0.598345,-0.962247,-0.342379,-0.506529,0.706748,-0.569281,0.447012,1.477272,-0.324896
4,0.586844,-0.489793,-0.647015,0.754353,2.061669,-0.271784,-0.629517,0.706748,0.103602,-1.095801,-1.155747,-0.466906


------------

# NN TREATEMENT ALONE

In [16]:


#X_train_nn =np.array(train['NN_dataframe'].tolist())
X_val_nn = np.array(val['NN_dataframe'].tolist())

## NN Normalize

### Divde columns by standard deviation

In [24]:

import numpy as np
means = []
std = np.loadtxt('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/params/std.csv', delimiter=',')

#X_train_nn_norm=X_train_nn.copy()
X_val_nn_norm = X_val_nn.copy()
for i in range(nb_param):
    #means.append(np.mean(X_train_nn[:,:,i]))
    #X_train_nn[:,:,i]=-means[i]
    #std.append(np.std(X_train_nn[:,:,i]))
    if std[i]!=0:
        #X_train_nn_norm[:,:,i]/= std[i]
        X_val_nn_norm[:,:,i]/= std[i]  

#savetxt('std.csv', std, delimiter=',')
#data = loadtxt('std.csv', delimiter=',')

In [25]:
amir_test = np.array([[2.1676032e+07, 8.6400000e+02, 1.5052800e+05, 4.0140800e+05,
        3.0000000e+00, 9.0000000e+00, 0.0000000e+00],
       [4.5760512e+07, 8.0000000e+02, 4.0140800e+05, 2.0070400e+05,
        3.2000000e+01, 9.0000000e+00, 0.0000000e+00],
       [6.8038656e+07, 6.2400000e+03, 2.0070400e+05, 7.5264000e+04,
        9.6000000e+01, 2.5000000e+01, 0.0000000e+00],
       [6.6006528e+07, 1.0512000e+04, 7.5264000e+04, 7.5264000e+04,
        1.4400000e+02, 2.5000000e+01, 1.0000000e+00],
       [6.6006528e+07, 1.0512000e+04, 7.5264000e+04, 7.5264000e+04,
        1.4400000e+02, 2.5000000e+01, 1.0000000e+00],
       [6.6006528e+07, 1.0512000e+04, 7.5264000e+04, 7.5264000e+04,
        1.4400000e+02, 2.5000000e+01, 1.0000000e+00],
       [3.6352512e+07, 1.2816000e+04, 7.5264000e+04, 3.1360000e+04,
        1.4400000e+02, 2.5000000e+01, 0.0000000e+00],
       [3.9544960e+07, 2.5200000e+04, 3.1360000e+04, 3.1360000e+04,
        2.4000000e+02, 2.5000000e+01, 1.0000000e+00],
       [3.9544960e+07, 2.5200000e+04, 3.1360000e+04, 3.1360000e+04,
        2.4000000e+02, 2.5000000e+01, 1.0000000e+00],
       [3.9544960e+07, 2.5200000e+04, 3.1360000e+04, 3.1360000e+04,
        2.4000000e+02, 2.5000000e+01, 1.0000000e+00],
       [2.4931200e+07, 3.4800000e+04, 3.1360000e+04, 1.5680000e+04,
        2.4000000e+02, 2.5000000e+01, 0.0000000e+00],
       [3.4825280e+07, 8.8800000e+04, 1.5680000e+04, 1.5680000e+04,
        4.8000000e+02, 2.5000000e+01, 1.0000000e+00],
       [3.4825280e+07, 8.8800000e+04, 1.5680000e+04, 1.5680000e+04,
        4.8000000e+02, 2.5000000e+01, 1.0000000e+00],
       [3.4825280e+07, 8.8800000e+04, 1.5680000e+04, 1.5680000e+04,
        4.8000000e+02, 2.5000000e+01, 1.0000000e+00],
       [3.7820160e+07, 9.6480000e+04, 1.5680000e+04, 1.8816000e+04,
        4.8000000e+02, 2.5000000e+01, 0.0000000e+00],
       [4.9015680e+07, 1.2499200e+05, 1.8816000e+04, 1.8816000e+04,
        5.7600000e+02, 2.5000000e+01, 1.0000000e+00],
       [4.9015680e+07, 1.2499200e+05, 1.8816000e+04, 1.8816000e+04,
        5.7600000e+02, 2.5000000e+01, 1.0000000e+00],
       [4.9015680e+07, 1.2499200e+05, 1.8816000e+04, 1.8816000e+04,
        5.7600000e+02, 2.5000000e+01, 1.0000000e+00],
       [3.3925248e+07, 1.8028800e+05, 1.8816000e+04, 9.4080000e+03,
        5.7600000e+02, 2.5000000e+01, 0.0000000e+00],
       [4.6183872e+07, 4.7116800e+05, 9.4080000e+03, 9.4080000e+03,
        1.1520000e+03, 2.5000000e+01, 1.0000000e+00],
       [4.6183872e+07, 4.7116800e+05, 9.4080000e+03, 9.4080000e+03,
        1.1520000e+03, 2.5000000e+01, 1.0000000e+00],
       [4.6183872e+07, 4.7116800e+05, 9.4080000e+03, 9.4080000e+03,
        1.1520000e+03, 2.5000000e+01, 1.0000000e+00],
       [5.8818816e+07, 6.0019200e+05, 9.4080000e+03, 1.5680000e+04,
        1.1520000e+03, 9.0000000e+00, 0.0000000e+00],
       [4.0140800e+07, 4.0960000e+05, 1.5680000e+04, 6.2720000e+04,
        3.2000000e+02, 1.0000000e+00, 0.0000000e+00],
       [1.6056320e+08, 1.6396800e+06, 6.2720000e+04, 6.2720000e+04,
        1.2800000e+03, 1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00]])

In [26]:
std

array([2.77884803e+08, 4.61103797e+05, 1.36894478e+05, 1.38975805e+05,
       3.11840030e+02, 1.21890588e+01, 3.94164309e-01])

In [27]:
for i in range(nb_param):
    #means.append(np.mean(X_train_nn[:,:,i]))
    #X_train_nn[:,:,i]=-means[i]
    #std.append(np.std(X_train_nn[:,:,i]))
    if std[i]!=0:
        #X_train_nn_norm[:,:,i]/= std[i]
       amir_test[:,i]/= std[i]  

In [28]:
amir_test

array([[7.80036612e-02, 1.87376466e-03, 1.09959146e+00, 2.88833009e+00,
        9.62031719e-03, 7.38367098e-01, 0.00000000e+00],
       [1.64674396e-01, 1.73496728e-03, 2.93224390e+00, 1.44416505e+00,
        1.02616717e-01, 7.38367098e-01, 0.00000000e+00],
       [2.44844826e-01, 1.35327448e-02, 1.46612195e+00, 5.41561892e-01,
        3.07850150e-01, 2.05101972e+00, 0.00000000e+00],
       [2.37531982e-01, 2.27974700e-02, 5.49795732e-01, 5.41561892e-01,
        4.61775225e-01, 2.05101972e+00, 2.53701306e+00],
       [2.37531982e-01, 2.27974700e-02, 5.49795732e-01, 5.41561892e-01,
        4.61775225e-01, 2.05101972e+00, 2.53701306e+00],
       [2.37531982e-01, 2.27974700e-02, 5.49795732e-01, 5.41561892e-01,
        4.61775225e-01, 2.05101972e+00, 2.53701306e+00],
       [1.30818640e-01, 2.77941758e-02, 5.49795732e-01, 2.25650788e-01,
        4.61775225e-01, 2.05101972e+00, 0.00000000e+00],
       [1.42307027e-01, 5.46514693e-02, 2.29081555e-01, 2.25650788e-01,
        7.69625375e-01, 2

# Add blocks with zeros

In [45]:


X_val_blocks = []
for sha in X_val_shape:
    ones = np.ones((sha, 1))
    zeros = np.zeros((37-sha,1))
    X_val_blocks.append(np.concatenate((ones,zeros)))
X_val_blocks = np.array(X_val_blocks)# Full Dataset (HW_params + NN arrays)

# Select power

In [51]:
X = ((X_val_nn_norm, X_val_hw_param_norm), y_val)
index = []
#X_small = [(),()]
for Y in X[1] :
    for i in range(len(Y)):
        #print(Y[i])
        if Y[i] > 40 and Y[i] < 65 :
            index.append(i)
X_restrained = ((X_val_nn_norm[index], X_val_hw_param_norm[index]), y_val[index])




In [52]:
len(X[1])

24274

In [53]:
len(X_restrained[1])

4208

# Full Dataset (HW_params + NN arrays)

In [54]:
#NORMED
#train_dataset_norm = tf.data.Dataset.from_tensor_slices(((X_train_nn_norm, X_train_hw_param_norm), y_train))
#test_dataset_norm = tf.data.Dataset.from_tensor_slices(((X_val_nn_norm, X_val_hw_param_norm), y_val))
test_dataset_norm = tf.data.Dataset.from_tensor_slices((X_restrained))

#train_dataset_norm = tf.data.Dataset.from_tensor_slices(( (X_train_blocks,X_train_nn_norm, X_train_hw_param_norm), y_train) )
#test_dataset_norm = tf.data.Dataset.from_tensor_slices(( (X_val_blocks,X_val_nn_norm, X_val_hw_param_norm), y_val) )

#train_dataset_norm = train_dataset_norm.shuffle(SHUFFLE_BUFFER_SIZE)
SHUFFLE_BUFFER_SIZE = 4
BATCH_SIZE = 128
test_dataset_norm = test_dataset_norm.shuffle(SHUFFLE_BUFFER_SIZE)

#train_dataset_norm = train_dataset_norm.batch(BATCH_SIZE)
test_dataset_norm = test_dataset_norm.batch(BATCH_SIZE)


# Get loss/model

In [47]:
def loss_1(target_power, predicted_power):
  return tf.math.abs((target_power - predicted_power)/target_power)

def loss_2(target_power, predicted_power):
  return tf.math.abs((target_power[:,0] - predicted_power[:,0])/target_power[:,0]) +tf.math.abs((target_power[:,1] - predicted_power[:,1])/target_power[:,1])

#def loss(target_power, predicted_power):
 # return tf.math.abs((target_power - predicted_power)/target_power)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=nb_training_batches,
    decay_rate=0.95)


optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)



## Models

In [48]:
last_layer = 128

multiple_power = True

if multiple_power ==True :

    checkpoint_path = '/Users/roxanefischer/Desktop/a_images/two_power/all_0.279_error/all'
    output_shape = 2
    #train_dataset = train_dataset_mult
    #test_dataset = test_dataset_mult
    loss = loss_2
else :
    checkpoint_path = '/Users/roxanefischer/Desktop/LSTM_3mult_32_117729_param_0.141_error/LSTM_3mult_32_117729_param'
    output_shape = 1
    #train_dataset = train_dataset_norm
    #test_dataset = test_dataset_norm
    loss = loss_1

#input_blocks = Input(shape=(max_blocks, 1),dtype='float32', name='blocks')
input_nn = Input(shape=(max_blocks, nb_param), dtype='float32', name='input_nn')
output_nn = layers.LSTM(128, return_sequences=True)(input_nn)
output_nn=layers.Dense(last_layer,  activation='relu')(output_nn)
#output_nn = tf.keras.layers.multiply([output_nn, input_blocks])
output_nn =tf.keras.layers.Lambda( lambda x: K.sum(x, axis=1))(output_nn)


model_hw = tf.keras.Sequential([
     layers.Dense(128, activation='relu', input_shape=(nb_hw_param,)),
     layers.Dense(last_layer , activation='linear')
 ])

concat = tf.keras.layers.multiply([output_nn, model_hw.output])
concat = tf.keras.layers.Concatenate()([concat,output_nn, model_hw.output])

output = tf.keras.layers.Dense(units=32, activation='relu')(concat)
output = tf.keras.layers.Dense(units=32, activation='relu')(output)
output = tf.keras.layers.Dense(units=output_shape, activation='relu')(output)
full_model = tf.keras.Model(inputs=[input_nn, model_hw.input], outputs=[output])



output_mult =tf.keras.layers.Lambda( lambda x: K.sum(x, axis=1))(output)
full_model_mult = tf.keras.Model(inputs=[input_nn, model_hw.input], outputs=[output_mult])




Wsave = full_model.get_weights()
print(f'PARAMS : {full_model.count_params()}')

PARAMS : 117762


In [49]:

full_model.load_weights(checkpoint_path)
#results = full_model.evaluate(test_dataset_norm)

In [51]:

if output_shape == 2:

    full_model_mult.compile(optimizer=optimizer, loss=[loss_1])
    def updateTargetModel(model, targetModel):
      modelWeights       = model.trainable_weights
      targetModelWeights = targetModel.trainable_weights

      for i in range(len(targetModelWeights)):
        targetModelWeights[i].assign(modelWeights[i])
    updateTargetModel(full_model, full_model_mult)

   # full_model_mult.evaluate((X_val_nn_norm, X_val_hw_param_norm), y_val_core)

In [52]:
full_model_mult.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_nn (InputLayer)           [(None, 37, 7)]      0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 37, 128)      69632       input_nn[0][0]                   
__________________________________________________________________________________________________
dense_25_input (InputLayer)     [(None, 12)]         0                                            
__________________________________________________________________________________________________
dense_24 (Dense)                (None, 37, 128)      16512       lstm_4[0][0]                     
____________________________________________________________________________________________

In [54]:
    full_model_mult.evaluate((X_val_nn_norm, X_val_hw_param_norm), y_core)

NameError: name 'y_core' is not defined

In [65]:
#%pycache


full_model.compile(optimizer=optimizer, loss=loss)

full_model.evaluate((test_dataset_norm))

t5= time.clock()


#print(f'Eval Full Model : {(t5-t4)/60} min')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
33/33 [==============================] - 1s 35ms/step - loss: 0.1292


In [66]:
full_model.evaluate((X_val_nn_norm, X_val_hw_param_norm), y_val)

759/759 [==============================] - 13s 17ms/step - loss: 0.1409


0.14092466235160828

In [ ]:
hw_centers = loadtxt('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/params/hw_centroids.csv', delimiter=',')
hw_centers_unscale = scaler.inverse_transform(hw_centers)

In [ ]:
# data_boths : puissance de 2
columns = ['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size', 'tot_mac', '1/mac_num', '1/mac_array_num', '1/data_bits', '1/sram_size', '1/max_filter_size', '1/tot_mac']
pd.DataFrame(hw_centers_unscale, columns=columns)

In [ ]:
ii=0
for ii in range(len(hw_centers)):
    print()
    print(f'index : {ii}')
    print(f'hw_center :{hw_centers[ii]}')
   # print(f'hw_center_unscale :{hw_centers_unscale[ii]}')
    hw_param_array = np.array([hw_centers[ii],]*X_val_hw_param_norm.shape[0])
    full_model.evaluate((X_val_nn_norm, hw_param_array), y_val)


In [35]:
ii=0
for ii in range(len(hw_centers)):
    print()
    print(f'index : {ii}')
    #print(f'hw_center :{hw_centers[ii]}')
   # print(f'hw_center_unscale :{hw_centers_unscale[ii]}')
    print(full_model.predict([nn_array_test, hw_centers[ii].reshape(1,12)], steps=1))


index : 0
[[49.722855]]

index : 1
[[57.656647]]

index : 2
[[60.51816]]

index : 3
[[66.16458]]

index : 4
[[39.812965]]

index : 5
[[37.970947]]

index : 6
[[66.056206]]

index : 7
[[84.150375]]

index : 8
[[40.023857]]

index : 9
[[60.4999]]


In [29]:

    nn_array_test= np.array([[7.66833971e-02, 1.81989163e-03, 1.11067407e+00, 2.91989295e+00,
          9.66273081e-03, 7.37706287e-01, 0.00000000e+00],
        [1.61887172e-01, 1.68508484e-03, 2.96179752e+00, 1.45994647e+00,
          1.03069129e-01, 7.37706287e-01, 0.00000000e+00],
        [2.41410695e-01, 1.31436618e-02, 1.48089876e+00, 5.47479928e-01,
          3.09207386e-01, 2.04918413e+00, 2.53786884e+00],
        [2.33511595e-01, 2.21420148e-02, 5.55337035e-01, 5.47479928e-01,
          4.63811079e-01, 2.04918413e+00, 2.53786884e+00],
        [2.33511595e-01, 2.21420148e-02, 5.55337035e-01, 5.47479928e-01,
          4.63811079e-01, 2.04918413e+00, 2.53786884e+00],
        [2.33511595e-01, 2.21420148e-02, 5.55337035e-01, 5.47479928e-01,
          4.63811079e-01, 2.04918413e+00, 2.53786884e+00],
        [1.28870709e-01, 2.69950592e-02, 5.55337035e-01, 2.28116637e-01,
          4.63811079e-01, 2.04918413e+00, 2.53786884e+00],
        [1.39898385e-01, 5.30801725e-02, 2.31390431e-01, 2.28116637e-01,
          7.73018465e-01, 2.04918413e+00, 2.53786884e+00],
        [1.39898385e-01, 5.30801725e-02, 2.31390431e-01, 2.28116637e-01,
          7.73018465e-01, 2.04918413e+00, 2.53786884e+00],
        [1.39898385e-01, 5.30801725e-02, 2.31390431e-01, 2.28116637e-01,
          7.73018465e-01, 2.04918413e+00, 2.53786884e+00],
        [8.83101622e-02, 7.33011906e-02, 2.31390431e-01, 1.14058318e-01,
          7.73018465e-01, 2.04918413e+00, 2.53786884e+00],
        [1.23201552e-01, 1.87044417e-01, 1.15695216e-01, 1.14058318e-01,
          1.54603693e+00, 2.04918413e+00, 2.53786884e+00],
        [1.23201552e-01, 1.87044417e-01, 1.15695216e-01, 1.14058318e-01,
          1.54603693e+00, 2.04918413e+00, 2.53786884e+00],
        [1.23201552e-01, 1.87044417e-01, 1.15695216e-01, 1.14058318e-01,
          1.54603693e+00, 2.04918413e+00, 2.53786884e+00],
        [1.33852023e-01, 2.03221232e-01, 1.15695216e-01, 1.36869982e-01,
          1.54603693e+00, 2.04918413e+00, 2.53786884e+00],
        [1.73402994e-01, 2.63277656e-01, 1.38834259e-01, 1.36869982e-01,
          1.85524432e+00, 2.04918413e+00, 2.53786884e+00],
        [1.73402994e-01, 2.63277656e-01, 1.38834259e-01, 1.36869982e-01,
          1.85524432e+00, 2.04918413e+00, 2.53786884e+00],
        [1.73402994e-01, 2.63277656e-01, 1.38834259e-01, 1.36869982e-01,
          1.85524432e+00, 2.04918413e+00, 2.53786884e+00],
        [1.20084070e-01, 3.79750720e-01, 1.38834259e-01, 6.84349910e-02,
          1.85524432e+00, 2.04918413e+00, 2.53786884e+00],
        [1.63384894e-01, 9.92447569e-01, 6.94171294e-02, 6.84349910e-02,
          3.71048863e+00, 2.04918413e+00, 2.53786884e+00],
        [1.63384894e-01, 9.92447569e-01, 6.94171294e-02, 6.84349910e-02,
          3.71048863e+00, 2.04918413e+00, 2.53786884e+00],
        [1.63384894e-01, 9.92447569e-01, 6.94171294e-02, 6.84349910e-02,
          3.71048863e+00, 2.04918413e+00, 2.53786884e+00],
        [2.08083593e-01, 1.26421805e+00, 6.94171294e-02, 1.14058318e-01,
          3.71048863e+00, 7.37706287e-01, 0.00000000e+00],
        [1.42006291e-01, 8.62763439e-01, 1.15695216e-01, 4.56233273e-01,
          1.03069129e+00, 8.19673652e-02, 0.00000000e+00],
        [8.87539318e-02, 5.39648421e-01, 4.62780863e-01, 7.12864490e-02,
          4.12276514e+00, 8.19673652e-02, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])
    nn_array_test = nn_array_test.reshape((1, *nn_array_test.shape))
    nn_array_test = tf.convert_to_tensor(nn_array_test, dtype=tf.float32)

    hw_array_test = np.array([1.023785822261557565e-01,-4.262310244296395600e-01,2.295995016604397698e-01,-4.441886177106757483e-01,-8.459185226109141587e-01,-2.938802963497538223e-01,-3.464242712811745339e-01,4.572050514380619490e-01,-6.563799113750447001e-01,3.803529224492657179e-01,1.081335456679819673e+00,-2.117873165907936950e-01])
    hw_array_test = hw_array_test.reshape((1, *hw_array_test.shape))
    hw_array_test = tf.convert_to_tensor(hw_array_test, dtype=tf.float32)